**PRESENTAZIONE DEL PROGETTO**

Questo progetto è un esperimento di traduzione automatica dall'inglese al cinese che confronta due diverse
tecniche di prompting per un Large Language Model (LLM).
1. Obiettivo: Verificare se una tecnica di prompting avanzata, chiamata Chain of Thought (CoT), produce traduzioni di qualità superiore rispetto a un modello baseline diretto e conciso.
2. Modello Utilizzato: mistralai/Mistral-7B-Instruct-v0.2.
3. Dataset: Il dataset scelto è un dataset standard per la traduzione inglese-cinese (iwslt2017) di 8549 rows, di cui viene usato un campione di 100 frasi per l'esperimento.
4. Metrica di Valutazione: Il punteggio BLEU (Bilingual Evaluation Understudy), una metrica standard per misurare la qualità di una traduzione automatica confrontandola
con una traduzione umana di riferimento. Un punteggio più alto indica una traduzione migliore.
5. Analisi automatica tra CoT e Non_CoT: I due modelli verranno messi a confronto su 100 campioni randomici e verranno valutati da un LLM giugice che decreterà chi dei due ha una media BLEU più alta.

**INSTALLAZIONE DELLE DIPENDENZE**:

●transformers: Permette di scaricare il modello pre-addestrato Mistral-7B e usarne il tokenizzatore.

●datasets: Mi permette di visualizzare e operare sul mio dataset per la traduzione.

●torch: Ne richiamo Pytorch per gestire i calcoli su GPU.

●pandas: Questa libreria è stata necessaria per la creazione di un DataFrame con i nomi aggiornati delle label del mio dataset.

●nltk e sacrebleu: Sono librerie per l'elaborazione del
linguaggio naturale. In questo specifico progetto, sono cruciali per la valutazione delle traduzioni del modelli.

●tqdm: Libreria di rilevanza minore, serve solo per la visualizzazione dei progressi nella parte dell'analisi automatica sui 100 campioni.

●numpy: Una libreria per il calcolo scientifico, usata qui per calcolare la media dei punteggi BLEU per decretare quale modello traduce meglio in media.

In [ ]:
!pip install transformers
!pip install -U datasets
!pip install torch
!pip install -U datasets fsspec huggingface_hub
!pip install pandas
!pip install torch
!pip install nltk
!pip install sacrebleu
!pip install tqdm
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 51.8 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.1

**IMPORTAZIONE DELLE LIBRERIE**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import pandas as pd
import torch, textwrap, time, re
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
import nltk
from tqdm import tqdm
import numpy as np

**AUTORIZZAZIONE ALL'USO DI MISTRAL TRAMITE TOKEN HUGGINGFACE**

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

**CARICAMENTO DEL MODELLO E DEL TOKENIZER**

In questa cella carico il modello "mistralai/Mistral-7B-Instruct-v0.2" e il suo tokenizer. All'interno della stessa cella il torch_dtype è stato impostato ad "auto" così che l'uso della memoria disponibile venga ottimizzato in base alla disponibilità.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype="auto"
)

cot_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


**APERTURA DEL DATASET**

Attraverso la libreria load_datasets apro il dataset e una volta visualizzato utilizzo un'altra libreria, ovvero pandas, per rinominare le label che volevo visualizzare con **'source_language'** (testo in lingua inglese) e **'target_language'** (testo in lingua cinese).
Creato il dataframe con le label rinominate ne prendo un sample di 100 campioni, facendo si che vengano scelti di volta in volta in forma randomica.

In [ ]:
ds = load_dataset("autoevaluate/autoeval-eval-iwslt2017-iwslt2017-zh-en-cdea8b-47199145201")

print (ds)
df = pd.DataFrame(ds['train'])


df = df.rename(columns={
    'source': 'source_language',
    'target': 'target_language'
})

subset_df = df.sample(n= 10).reset_index(drop=True)
print(subset_df)



README.md:   0%|          | 0.00/862 [00:00<?, ?B/s]

predictions.parquet:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8549 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['target', 'source', 'evaluation_predictions'],
        num_rows: 8549
    })
})
                                     target_language  \
0  话说，我是学应用数学的 对于每一个学应用数学的人来说 有一个特殊的难题，就是 我们和管理顾问都很像   
1  对她来说，尽管悲催的灾难和种种艰辛，那些照片 是给他儿子最好的礼物， 这些照片让他可以重新翻...   
2                      可是如何才能转变它的角色 从而让数字信息进入我们的世界中？   
3                            我现在就给你们看100亿个像素给你们提供了什么   
4                               或“去阿尔巴尼亚好玩么？”等类似的问题。   
5                      用诗人的话说， 结婚并同床， 然后她生下更多的儿子和女儿，   
6                  我的病患也不是车子， 他们不会每次都用相同的方式去描述他们的症状。   
7                                    那是我人生中最低落的时刻之一。   
8  我们能用它 让人们对人类自己 产生更多的同感 让人们真真正正地 体会到 穿着别人的鞋走一英里的感受   
9                                  这一切都是有可能的，只有你有手机。   

                                     source_language  \
0  So, well, I do applied math,  for anyone who d...   
1  For her, despite all of this, those photos  we...   
2  But how about reversing its role  and having t...   
3  So I'm going to try an

**DEFINIZIONE DEL PROMPT PER LA CHAIN OF THOUGHT**

Questa cella è una delle più importanti del codice in quanto rapresenta la creazione del prompt su cui si creerà la Chain of Thought. Questo dovrebbe infatto migliorare la performance traduttiva del modello

*   Il prompt è stato dato in lingua inglese in quanto è la lingua a cui più è stato esposto ed è composto principalmente dalle seguenti istruzioni:


1.   Dichiarazione del ruolo di traduttore esperto dalla lingua inglese a quella cinese.
2.   Il modello deve trasporre la proposizione data in inglese in cinese.
3.   Generare un output per volta.
4.   Se non certo della traduzione deve continuare a stampare i ragionamenti chiamandoli 'Thought'.
5.   Restituire i pensieri solo in lingua inglese: questo è stato necessario in quanto nei primi tentativi tendeva a restituire la catena di pensiero in italiano o in cinese.
6.   Se stampa due volte lo stesso Thought deve restituire la final answer: questo è stato aggiunto quasi alla fine quando più di un campione ha fatto allucinare il modello stampando all'infinito lo stesso pensiero.
7.   Restituire la final answer solo in lingua cinese: è stato necessario aggiungere questa istruzione in quanto molte frasi contenevano nomi e appelativi tipici delle lingue occidentali che non venivano tradotte in maniera sistematica ma opzionale.
8.   Stampare la final answer utilizzando il prefisso 'FINAL': essenziale per poter fare il paragone con la traduzione che ci aspettavamo di trovare come inferenza della CoT.
9.   Infine per essere certa di ottenere i prefissi 'Thought' e 'FINAL' ho ribadito di fare attenzione affinchè tutti gli output avessero questi prefissi.

In [ ]:

SYSTEM_TRANSLATOR_MSG = textwrap.dedent("""
    You are an expert translator translating from English to Chinese.
    Think step by step before providing the final translation.

    Instructions:
    - Translate the given English sentence into Chinese.
    - translate each line of the text given.
    - If you are still reasoning or analyzing the sentence, write exactly:
      Thought: <your thought or analysis>
    - Provide the Thoughts only in English.
    - If you provide the same thought two times provide the final translation.
    - Provide the final translation in Chinese.
    - When and only when the translation is complete, write exactly:
      FINAL: <translated sentence in Chinese>

    Make sure the output always starts with "Thought:" or "FINAL:".
""")


**ONE_STEP**
La funzione **one_step** gestisce una singola interazione con il modello. Prende una cronologia di messaggi (messages), la formatta in un prompt, la invia al modello e restituisce la risposta generata.   

Parametri istanziati:
*   max_tokens: definisco qui il massimo dei token utilizzabili per fornire la risposta.
*   temperature: questo è un valore importantissimo per il task che deve essere svolto dal modello. Definisce infatti quanto il modello può essere creativo, scegliendo magari parole che solitamente non occorrono in un determinato contesto. Nel mio caso ho deciso di usarla a 0.3, limitando un pò la creatività del modello a favore di una traduzione più rigida.
*   do_sample=True: indica che il modello deve campionare dalla distribuzione dei token, invece di prendere sempre quello più probabile e questo giova un ruolo fondamentale se si vuole usare appunto la temperatura.
*   pad_token_id: qui si indica come usare il padding.
*    eos_token_id: determino l'end of sequnece.


In [ ]:
def one_step(messages, max_tokens=200, temperature=0.3):
    prompt = "".join(f"<|{m['role']}|>\n{m['content']}\n" for m in messages) + "<|assistant|>\n"

    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_new_tokens= max_tokens,
            temperature=0.3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result[len(prompt):].strip().split("\n")[0]

**FUNZIONE CHAIN OF THOUGHT**
1. Inizia con il prompt di sistema.
2. Entra in un ciclo (while True): All'inizio non era stato scelto un ciclo ma erano stati determinati degli step massimi, ciò non era sufficiente per fargli restituire la final answer. Ho quindi usato un ciclo che fa stampare i pensieri fino a quando il modello effettivamnete trova la risposta che ritiene essere quella finale. Durante l'esecuzione del ciclo:


*   Chiama la funzione one_step per ottenere il prossimo pezzo di testo dall'assistente (un "Thought" o il "FINAL").
*   Controlla se la risposta inizia con FINAL:. Se sì, ha trovato la risposta definitiva, la estrae e interrompe il ciclo.

*   Se non è la risposta finale, la considera un "pensiero", la aggiunge alla cronologia della conversazione e ripete il ciclo.

3. Alla fine, restituisce sia i Thoghts raccolti in (scratchpad) sia la risposta finale.

In [ ]:
def chain_of_thought(user_query, max_steps=10, stream=False):
    messages = [
        {"role": "system", "content": SYSTEM_TRANSLATOR_MSG},
        {"role": "user", "content": user_query}
    ]
    scratchpad = []

    answer = "(nessuna risposta definitiva)"
    while True:
        assistant_line = one_step(messages)

        if stream:
            print(assistant_line)
            time.sleep(0.5)

        if assistant_line.startswith("FINAL:"):
            answer = assistant_line.removeprefix("FINAL:").strip()
            break


        if not assistant_line.startswith("Thought:"):
            assistant_line = "Thought: " + assistant_line

        scratchpad.append(assistant_line)
        messages.append({"role": "assistant", "content": assistant_line})

    return scratchpad, answer


**FUNZIONE JUDGE_TRANSLATION**

1. Prende due stringhe in input: reference (la traduzione corretta, dal dataset) e hypothesis (la traduzione generata dal nostro modello).

2. Usa la funzione sentence_bleu della libreria nltk per calcolare il punteggio BLEU. Questo punteggio misura la somiglianza tra le due frasi basandosi sulla sovrapposizione di sequenze di parole (n-grammi). Il valore va da 0 (nessuna somiglianza) a 1 (corrispondenza perfetta).

3. Infine ho convertito il tutto in un giudizio qualitativo:"Ottimo", "Buono", "Medriocre" e "Scarso".

In [ ]:
def judge_translation(reference, hypothesis):
    """
   Evaluate the quality of a hypothetical translation against a reference using the BLEU score.

    Args:
        reference (str): The reference sentence.
        hypothesis (str): The translated sentence to be evaluated.

    Returns:
        tuple: BLEU score (float) and qualitative assessment (str).
    """

    reference_tokens = [list(reference)]
    hypothesis_tokens = list(hypothesis)

    smoothie = SmoothingFunction().method1
    bleu_score = sentence_bleu(reference_tokens, hypothesis_tokens, smoothing_function=smoothie)

    if bleu_score > 0.75:
        judgment = "Ottimo"
    elif bleu_score > 0.5:
        judgment = "Buono"
    elif bleu_score > 0.25:
        judgment = "Mediocre"
    else:
        judgment = "Scarso"

    return bleu_score, judgment



**ESECUZIONE DEL TEST QUALITATIVO DEL MODELLO COT**

In questa funzione per ogni riga del subset di 100 campioni formula una richiesta di traduzione:
1. Si richiama la funzione chain_of_thought per elaborare e mettere in atto i parametri.
2. Incrementa il numero degli esempi affinchè sia più chiaro da leggere.
3. Restituisce i ragionamenti.
4. Restituisce la traduzione attesa dal modello per poterla paragonare con quella finale.
5. Infine calcola di BLUE score.

Questa cella in realtà serve solo per vedere come il codice di comporta ed è stata essenziale per la visualizzazione delle allucinazioni e per la visualizzazione dell'errore iniziale che appunto restituiva una metrica BLEU pari a 0 a causa della mancata restituzione della final answer da parte del modello a causa degli step predefiniti.

In [ ]:
if __name__ == "__main__":
    for idx, row in subset_df.iterrows():
        question = f"Translate the given English sentence into Chinese: {row['source_language']}"

        print(f"\n=== ESEMPIO {idx+1} ===")
        thoughts, answer = chain_of_thought(question, stream=True, max_steps=3)

        print("TRADUZIONE FINALE:", answer)

        print("TRADUZIONE ATTESA :", row['target_language'])

        bleu, giudizio = judge_translation(row['target_language'], answer)
        print(f"BLEU Score: {bleu:.4f}")
        print("Giudizio del LLM Giudice:", giudizio)

    print("\n=== FINE ===")


=== ESEMPIO 1 ===
Thought: The speaker is comparing the field of applied math to management consulting.
Thought: In English, "So, well, I do applied math" can be translated to "我做应用数学" in Chinese.
Thought: The phrase "for anyone who does applied math" can be translated to "对于做应用数学的人" in Chinese.
Thought: The comparison "we are like management consultants" can be translated to "我们像管理咨询师一样" in Chinese.
FINAL: 我做应用数学，对于做应用数学的人，我们像管理咨询师一样。
TRADUZIONE FINALE: 我做应用数学，对于做应用数学的人，我们像管理咨询师一样。
TRADUZIONE ATTESA : 话说，我是学应用数学的 对于每一个学应用数学的人来说 有一个特殊的难题，就是 我们和管理顾问都很像
BLEU Score: 0.1573
Giudizio del LLM Giudice: Scarso

=== ESEMPIO 2 ===
Thought: Understanding the context of the sentence, it seems to be expressing that the photos hold significant value for him, serving as a reminder of the past that isn't tainted by the negative events that occurred in March.
Thought: Translating the sentence into Chinese requires understanding each part separately and then combining them.
Thought: For her: 对她来说
Thoug

**FUNZIONE PER LA BASELINE_RESPONSE**

Qui definisco il termine di paragone al modello che avrà il prompt, ovvero il modello baseline. Questo modello non ragionerà passo passo, la sua inferenza sarà dunque breve.


1.   Richiama one_step per ottenere la traduzione.
2.   Non richiama la funzione chain_of_thought in quanto non agirà secondo le sue regole.



In [ ]:
def baseline_response(question, max_tokens=60):
    messages = [
        {"role": "system", "content": "You are a translator. Translate the following sentences from English to Chinese and do so as concisely as possible."},
        {"role": "user", "content": str(question).strip()}
    ]
    return one_step(messages, max_tokens=max_tokens)


translations = []

for text in subset_df["source_language"].astype(str):
    translation = baseline_response(text)
    translations.append(translation)

print(translations)


['我们做应用数学，任何做应用数学的人，就像管理咨询师一样。', '她对此，但是，那些照片仍然是他最好的回礼，他可以再次看到的东西，他记忆中从那个在三月份整个他人生活都发生了变化或被破', '反过来，数字信息直接到我们那里来吧？', '我试图用十亿像素所给出的实际情况展示给你。', '你的阿尔巴尼亚旅行如何？(Nǐ de Ālbaniá lǚrén rú hé?)', '她是一位诗人女子，她娶了，睡在了，然后生下更多的儿女。', '患者不像汽车，他们的症状表达方式不一致。', '这是我生命中的最低点。', '我认为增强现实可以作为鼓励人类之间更多同情心的方式，通过直接显示给某人，另一个人穿过的鞋子所看到的景象。', '通过手机实现所有这些事情。']


**CONFRONTO AUTOMATICO TRA COT E NON_COT SU 100 CAMPIONI**

Questa è la funzione che ci darà modo di vedere quale modello traduce meglio ottenendo la media BLUE più alta.



1.   Mi inizializzo due liste che conterranno gli score dei due modelli.
2.   Incomincia poi a iterare sulle 100 proposizioni e per visuallizzare il progresso di tale operazione tramite la libreria tqdm visualizzo la barra di progresso.
3. Prima fa tradurre al modello CoT e calcola il punteggio BLUE aggiungendolo alla prima lista.
4. In seguito fa la medesima operazione con il modello Non_CoT.
5. Finiti questi due cicli calcola la media dei punteggi BLEU ottenuti utilizzando la libreria numpy.
6. Alla fine decreterà chi dei due in media ha tradotto meglio.


In [ ]:
cot_bleu_scores = []
noncot_bleu_scores = []

if __name__ == "__main__":
    print("== INIZIO VALUTAZIONE ==")

    for idx, row in tqdm(subset_df.iterrows(), total=len(subset_df), desc="Valutazione esempi"):
        question = f"Traduci la seguente frase dall'inglese al cinese: {row['source_language']}"
        expected = row['target_language']

        #Chain of Thought (CoT)
        thoughts, cot_answer = chain_of_thought(question, stream=False, max_steps=5)
        bleu_cot, _ = judge_translation(expected, cot_answer)
        cot_bleu_scores.append(bleu_cot)

        #Baseline
        noncot_answer = baseline_response(question)
        print("Non-CoT Answer:", noncot_answer)
        bleu_noncot, _ = judge_translation(expected, noncot_answer)
        noncot_bleu_scores.append(bleu_noncot)

    avg_cot_bleu = np.mean(cot_bleu_scores)
    avg_noncot_bleu = np.mean(noncot_bleu_scores)

    print("\n== RISULTATI FINALI ==")
    print(f"Media BLEU (CoT): {avg_cot_bleu:.4f}")
    print(f"Media BLEU (Non-CoT): {avg_noncot_bleu:.4f}")

    if avg_cot_bleu > avg_noncot_bleu:
        print("Il modello CoT ha tradotto meglio in media.")
    elif avg_noncot_bleu > avg_cot_bleu:
        print("Il modello Non-CoT ha tradotto meglio in media.")
    else:
        print("Le prestazioni dei due modelli sono equivalenti.")


== INIZIO VALUTAZIONE ==


Valutazione esempi: 100%|██████████| 100/100 [33:01<00:00, 19.82s/it]


== RISULTATI FINALI ==
Media BLEU (CoT): 0.0978
Media BLEU (Non-CoT): 0.0896
Il modello CoT ha tradotto meglio in media.


**OSSERVAZIONI SUL PROGETTO**

I risultati ottenuti dall'esecuzione della valutazione automatica hanno mostrato come il prompting nella Chain of Thought migliori le prestazioni finali del modello. Mi sento di dire che questo tipo di osservazioni siano più evidenti in task quali quelli traduttivi.

Il progetto si può considerare completo, al suo interno però non troviamo la self consistency. Questo è dovuto al fatto che avrebbe richiesto un dispendio computazionale non indifferente per rendere il modello stocastico per agire in forma statistica correttamente. Ho provato ad utilizzare modelli quantizzati di Mistral per conservare unità di calcolo per provare a farlo ma non giravano bene.

Ho avuto dei dubbi per l'uso della metrica BLEU in quanto all'inizio a causa dell'uso di modelli quantizzati ho avuto difficoltà con i tokenizer ma una volta riuscita ad aprire il modello 7B il tokenizer ha funzionato bene riuscendo a farmi usare BLEU. La maggior parte delle traduzioni ha avuto uno score sotto lo 0,25, questo è dato dal fatto che la metrica è molto severa anche nella sintassi e calcolava come errore anche la non trasposizione di termini tipicamente occidentali.

In generale mi ritengo soddisfatta del risultato, sia per la sua difficoltà, sia per le cose che ho imparato svolgendolo, soprattutto negli errori e impedimenti.



